# RNN


In [2]:
%%bash
git add theoretical_consideraiton.ipynb

In [3]:
git commit -a -m "added theoretical_consideration"

SyntaxError: invalid syntax (<ipython-input-3-981629d018e7>, line 1)

$$
\begin{align}
\text{時点$t$における$l$番目の(hidden) layerにおけるnode value}=h^{(l)}_t=f\left(
W^{(l)} \left[
a
\right]
\right
\end{align}
$$


* RNNはoption on optionのように考えると理解しやすい

対応は以下のような感じか？

* ひとつめのオプションに対してクーポンが入力される
    * クーポンはembedded vectorになっている「ワード」なので300次元とかのイメージ 
    * なのでそれを受ける一つ目のオプションも300次元の入力を食えるようになっている
    * 300次元は株価方向のラティスをイメージ？
* ひとつめのオプションバリュー自体も300次元であれば株価イメージでいけるがかならずしも同じ次元である必要はない（よね？）
    * その場合の対応するイメージは？？例えば200次元であれば、オプションンバリューのレイヤーは株価方向を粗視化しているとみなすのがいいか？
        *そうだとするとそれってオプションプライシングでも応用可能かも？


# gated RNN

$$

$$

# GRU
GRU の解説は
https://mosko.tokyo/post/pytorch-rnn/

上記文献の場合、
* candidate node $n$が一番素朴なRNN(活性化関数を$\tanh$としたもの)と自然に対応が付くノード
    * ただし、RNNで素直に一時点前の同じレイヤーの隠れ層からの伝搬があるのに対して、GRUではreset gate $r$がゼロの時は伝搬を遮断する
    * そのreset gateは普通のRNNスタイル（活性化関数はsigmoid）のノード
         * ただし、一時点前のreset gateからの伝搬があるわけではなく、通常のhidden node $h$からの伝搬
* update gate $z$が1になるときは　candidate nodeの値は捨てられて、一時点前の同じレイヤーがそのままスライド採用される
    * これはGated RNNに由来する方法か
    

RNNの設計のうち、オプション的な解釈がむずかしいかもしれないところ
* ひとつめのオプションの現在価値と二つ目のオプションの1ステップ前（すなわち未来）とを混合してから活性化関数にかける
    * ひとつめのオプション価値に依存するクーポンと思えばよいか